Our problematic is that density is really respected in our network as long as it evolves.  
We must, therefore, find rules such that it can work properly, having nice and realistic 2D representation.

In [1]:
from gpn5 import GrowingPlanarNetwork
import networkx as nx
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from heapq import heappush, heappop

In [6]:
class RGPN(GrowingPlanarNetwork):
    def init_square(self, size=3):
        super().init_square(size=size)
        total_size = size**2 - 1
        # set north
        for i in range(size):
            self.G.nodes[i]["side"] = self.G.nodes[i].get("side", set()) | {"N"}
        
        # set south
        for i in range(size):
            self.G.nodes[total_size - i]["side"] = self.G.nodes[total_size - i].get("side", set()) | {"S"}
        
        # set west
        for i in range(size):
            self.G.nodes[i * size]["side"] = self.G.nodes[i * size].get("side", set()) | {"W"}
        
        # set east
        for i in range(size):
            self.G.nodes[total_size - i * size]["side"] = \
                self.G.nodes[total_size - i * size].get("side", set()) | {"E"}
            
    def diffuse_border(self, node):
        # if the border is a corner, we can just return
        # ideally, we shall never have a length of 3 ...
        
        if self.is_border_node(node):
            if len(self.G.nodes[node].get("side", set())) >= 2:
                return
            ngb1, ngb2 = self.get_border_neighbours(node)
            side1 = self.G.nodes[ngb1].get("side", set())
            side2 = self.G.nodes[ngb2].get("side", set())
            side = self.G.nodes[node].get("side", set())
            assert len(side1 & side2) == 1, f"side is not of the right size, " \
                f"should be one : {side1}, {side2}, {ngb1}, {ngb2}" \
                f" with node {node}, {side}"
            self.G.nodes[node]["side"] = side1 & side2
            
        else:
            if "side" in self.G.nodes[node]:
                del self.G.nodes[node]["side"]
    
    def update_corner(self, node):
        if self.is_border_node(node):
            ngb1, ngb2 = self.get_border_neighbours(node)
            side1 = self.G.nodes[ngb1].get("side", set())
            side2 = self.G.nodes[ngb2].get("side", set())
            side_ref = self.G.nodes[node].get("side", set())
            
            if len(side1 & side2) == 0:
                self.G.nodes[ngb1]["side"] = side1 | side2
            
    # TODO override duplicate to set the border (NSEW)
    def duplicate_node(self, node):
        new_node = super().duplicate_node(node)
        self.diffuse_border(new_node)
            
    
    # TODO override remove to check for corners
    def remove_node(self, node):
        self.update_corner(node)
                
        super().remove_node(node)
    
    def edge_swapper(self):
        pass
    
    def update_all_dist(self):
        self.update_dist(direction="N")
        self.update_dist(direction="S")
        self.update_dist(direction="E")
        self.update_dist(direction="W")
        
    def mean_NS(self):
        return np.mean([self.G.nodes[n]["dist_N"] + self.G.nodes[n]["dist_S"] for n in self.G.nodes])
    
    def mean_EW(self):
        return np.mean([self.G.nodes[n]["dist_E"] + self.G.nodes[n]["dist_W"] for n in self.G.nodes])
    
    def update_dist(self, direction="N"):
        """
        Update for each node its distance to each sides
        Direction is one of NSEW
        """
        key = "dist_" + direction
        queue = []
        nodes = set(self.G.nodes)
        # NORTH
        for i in self.G.nodes:
            n = self.G.nodes[i]
            if direction in n.get("side", set()):
                value = 0
            else:
                value = np.inf
                
            n[key] = value
            heappush(queue, (value, i))
            
        while queue and nodes:
            _, inode = heappop(queue)
            if inode not in nodes:
                continue
                
            nodes.remove(inode)
            node = self.G.nodes[inode]
            value = node[key] + 1
            for ngb in node["ngb"]:
                if value < self.G.nodes[ngb][key]:
                    self.G.nodes[ngb][key] = value
                    heappush(queue, (value, ngb))

    
    def show_dist(self, k=5, iterations=1000, figsize=(10, 10), pos=None):
        # green for NS, red for EW
        plt.figure(figsize=figsize)
        if not pos:
            pos = nx.spring_layout(self.G, k=k, iterations=iterations)
        green = lambda x: format(max(0, 255 - (x["dist_N"] + x["dist_S"]) * 20), '02x')
        red = lambda x: format(max(0, 255 - (x["dist_E"] + x["dist_W"]) * 20), '02x')
        corner = lambda x: len(x.get("side", set())) >= 2
        def col_border(node):
            side = tuple(node.get("side", set()))
            return {
                ("N",): "#888822",  # yellow
                ("S",): "#228888",  # cyan
                ("E",): "#882288",  # purple
                ("W",): "#666666",  # grey
            }.get(side, "")
        colors = ["#000000" if corner(self.G.nodes[n]) else 
                  col_border(self.G.nodes[n])
                  or "#" + red(self.G.nodes[n]) + green(self.G.nodes[n]) + "ff" 
                  for n in self.G.nodes]
        nx.draw_networkx(self.G, pos, node_color=colors)
        
    def _reduce_on_axis(self, n, axis):
        """
        Warning, non node-cycle reducing, taking one step further
        Rationale is that empirically, we did not find much into cycle opportunity
        But we will have to do it later
        """
        assert axis in ["NS", "EW"], "Wrong axis argument"
        
        if self._quick_reduce(n, axis):
            print("Quick reducing", n)
            return True
        
        distA, distB = ("dist_N", "dist_S") if axis == "NS" else ("dist_E", "dist_W")
        node = self.G.nodes[n]
        N, S = node[distA], node[distB]
        ingbs = set(self.get_intermediate_neighbours(n)) - {-1}
        
        for ingb in ingbs:
            dngbs = self.ngb(ingb, net="dual")
            
            for edge_dngb in dngbs:
                node_dngb = self.get_other_node(edge_dngb, ingb)
                if node_dngb in (set(ingbs) | {-1}):
                    continue
                    
                pngbs = self.get_intermediate_neighbours(node_dngb, net="dual")
                
                # the best thing would be to make a list of possible
                # candidates then pick the very best, the same way as 
                # _augment
                for pngb in pngbs:
                    # no edge
                    if (n, pngb) in self.G.edges:
                        continue

                    pnode = self.G.nodes[pngb]
                    pN, pS = pnode[distA], pnode[distB]

                    # nothing to win (here for NS but works also for EW)
                    if pN >= N - 1 and pS >= S - 1:
                        continue

                    # not killing
                    if False:  # osef for now
                        continue

                    # here we are -1 safe
                    # print("Dual edge", edge_dngb, ingb, node_dngb)
                    self.replace_edge(self.dual(edge_dngb), (n, pngb))
                    # THERE we added True
                    return True
                
    def _get_border_candidates(self, node):
        assert self.is_border_node(node)
        return [n for ngb in gpn.get_border_neighbours(node)
                for n in gpn.get_border_neighbours(ngb)
                if n != node]
    
    def _get_quick_candidates(self, node, dual_node):
        assert dual_node != -1, "Operation not allowed for dual -1"
        return [ngb for ngb in 
                self.get_intermediate_neighbours(dual_node, net="dual")
                if ngb != node]
    
    def _get_common_border_ngb(self, n_a, n_b):
        s_a = set(gpn.get_border_neighbours(n_a))
        s_b = set(gpn.get_border_neighbours(n_b))
        return list(s_a & s_b)[0]
                
    def _quick_reduce(self, n, axis):
        """
        The idea is the same as "_reduce_on_axis" but here
        we just create a direct edge without removing anything
        directly (maybe something will be removed through
        stabilize_ngb later)
        """
        assert axis in ["NS", "EW"], "Wrong axis argument"
        
        A, B = axis
        node = self.G.nodes[n]
        N, S = self._dist(n, A), self._dist(n, B)
        dist = self._dist(n, axis)
        # get dual nodes
        ingbs = self.get_intermediate_neighbours(n)
        
        # gather all candidates without any filter
        candidates = []
        for ingb in ingbs:
            if ingb == -1:
                ls_c = self._get_border_candidates(n)
            else:
                ls_c = self._get_quick_candidates(n, ingb)
            candidates += [(c, ingb) for c in ls_c]
                
        # filter candidates
        filtered_candidates = [(c, d) for c, d in candidates
                               if self._dist(c, A) < N - 1
                               or self._dist(c, B) < S - 1]
        
        # pick best
        random.shuffle(filtered_candidates)
        
        # TODO number of ngb does not correct for already ngb with the next removed edge
        # neither border sides
        filtered_candidates.sort(key=lambda x: len(self.ngb(x[0])))

        # if empty
        if len(filtered_candidates) == 0:
            return False
            
        # select the best one (the first)
        candidate, dual = filtered_candidates[0]
        self.build_new_edge(n, candidate, dual)
        
    def build_new_edge(self, n, candidate, dual):
        if dual == -1:
            ref_node = self._get_common_border_ngb(n, candidate)
            if self.is_corner(ref_node):
                self.update_corner(ref_node)
            print("Got our guilty line ?")
            self._make_border_edge(ref_node, (n, candidate))
            print(locals())
            self.diffuse_border(ref_node)
        else:
            self._make_edge(dual, (n, candidate))
            
        return True
                
    def replace_edge(self, edge1, edge2):
        print("Shall replace here", edge1, edge2)
        dual_node = self._remove_edge(edge1)
        # print("Dual node", dual_node)
        self._make_edge(dual_node, edge2)
        
    def shorten_dist(self):
        """
        For now, one max action per round, otherwise tricky mechanisms could occur
        """
        self.update_all_dist()
        mean_NS = self.mean_NS()
        mean_EW = self.mean_EW()
        # find nodes with most pressure
        # for threshold, let's say *1.5 compared to mean
        nodes = list(self.G.nodes)
        random.shuffle(nodes)
        # TODO we must sort from the most "out" to the less
        # list candidates
        candidates = list()
        for n in nodes:
            node = self.G.nodes[n]
            val_NS = (node["dist_N"] + node["dist_S"]) / mean_NS
            val_EW = (node["dist_E"] + node["dist_W"]) / mean_EW
            if val_NS >= 1.1:
                candidates.append((n, "NS", val_NS))
                
            if val_EW >= 1.1:
                candidates.append((n, "EW", val_EW))
            
        # pick best candidate
        candidates.sort(key=lambda x: x[2], reverse=True)
        for n, axis, _ in candidates:
            if self._reduce_on_axis(n, axis=axis):
                self.check_all()
                self.stabilize(n)
                return True
        
        # success is implicit
        return False
                
    def maxen_dist(self):
        self.update_all_dist()
        mean_NS = self.mean_NS()
        mean_EW = self.mean_EW()
        # find nodes with most pressure
        # for threshold, let's say *1.5 compared to mean
        nodes = list(self.G.nodes)
        random.shuffle(nodes)
        candidates = list()
        for n in nodes:
            node = self.G.nodes[n]
            val_NS = (node["dist_N"] + node["dist_S"]) / mean_NS
            val_EW = (node["dist_E"] + node["dist_W"]) / mean_EW
            if val_NS <= 0.9:
                candidates.append((n, "NS", val_NS))
                
            if val_EW <= 0.9:
                candidates.append((n, "EW", val_EW))
            
        # pick best candidate
        candidates.sort(key=lambda x: x[2], reverse=False)
        for n, axis, _ in candidates:
            if self._augment(n, axis=axis):
                self.stabilize(n)
                return True
        
        # success is implicit
        return False
                
    def setup_edge(self, edge, dual_node, candidate):
        print("Setup", edge, dual_node, candidate)
        # setup the edge and delete the one chosen
        node1, node2 = edge
        if (node1, candidate) not in self.G.edges:
            other_dual = self._make_edge(dual_node, (node1, candidate))
            # update dual_node
            ingb = set(self.get_intermediate_neighbours(dual_node, net="dual"))
            if not ingb.issuperset({node2, candidate}):
                dual_node, other_dual = other_dual, dual_node
            
        if (node2, candidate) not in self.G.edges:
            self._make_edge(dual_node, (node2, candidate))
        
        self._remove_edge(edge)
        self.diffuse_border(candidate)
    
    # TODO, find a better way
    def _dist(self, n, axis):
        node = self.G.nodes[n]
        return sum([node["dist_" + str(x)] for x in axis])
                
    def _augment(self, n, axis="NS"):
        A, B = axis
        node = self.G.nodes[n]
        N, S = self._dist(n, A), self._dist(n, B)
        dist = self._dist(n, axis)
        target_node = None
        # first step, find ngb that has the same dist
        for ngb in self.ngb(n):
            if self._dist(ngb, axis) <= dist:
                target_node = ngb
                break
                
        if target_node == None:
            raise RuntimeError(f"Should not have no ngb with same dist, dist is {dist}")
            
        dual_nodes = set(self.dual((n, target_node))[:2]) - {-1}
        
        # build list of candidate link node
        candidate_list = []
        for dual_node in dual_nodes:
            pngbs = set(self.get_intermediate_neighbours(dual_node, net="dual")) - {n, target_node}
            candidate_list += [(dual_node, pngb) for pngb in pngbs 
                               if self._dist(pngb, A) >= N and self._dist(pngb, B) >= S]
            
        random.shuffle(candidate_list)
        # TODO number of ngb does not correct for already ngb with the next removed edge
        candidate_list.sort(key=lambda x: len(self.ngb(x[1])))

        # if empty
        if len(candidate_list) == 0:
            return False
            
        # select the best one (the first)
        dual, candidate = candidate_list[0]
        
        self.setup_edge((n, target_node), dual, candidate)
        return True

In [7]:
seed = 1
random.seed(seed)
np.random.seed(seed)
gpn = RGPN()
gpn.init_square(6)
gpn.update_all_dist()

def p_dupl():
    return np.random.random() > 0.48

In [8]:
seed = 1
random.seed(seed)
np.random.seed(seed)
gpn = RGPN()
gpn.init_square(8)
gpn.update_all_dist()
ignore_export = True

def quick_export(gpn, pos, name):
    global ignore_export
    if ignore_export:
        return
    gpn.update_all_dist()
    gpn.show_dist(pos=pos, figsize=(8, 8))
    plt.savefig(name)
    plt.close()

pos = nx.spring_layout(gpn.G, k=0.1, iterations=50)
root = "output/evonet6/"
ratio = 0
for i in range(300):
    # print(gpn.G.number_of_nodes(), ratio)
    # gpn.check_all()
    if p_dupl():
        ratio += 1
        gpn.duplicate_random_node()
    else:
        ratio -= 1
        gpn.remove_random_node()
        
    pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=10)
    quick_export(gpn, pos, root + str(i).zfill(4) + "_a.png")
    if i == 31:
        gpn.check_all()
    if gpn.shorten_dist():
        pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=10)
        quick_export(gpn, pos, root + str(i).zfill(4) + "_b.png")
    if i == 31:
        gpn.check_all()
    if gpn.maxen_dist():
        pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=10)
        quick_export(gpn, pos, root + str(i).zfill(4) + "_c.png")
        
    print(f"Run {i}")
    gpn.check_all()
        
    if i % 100 == 0:
        print()
        print("Iteration", i, "ratio", ratio, "density", gpn.density())

Run 0

Iteration 0 ratio -1 density 0.08035714285714286
Shall replace here (12, 11) (20, 4)
Setup (59, 51) 45 60
Run 1
Shall replace here (21, 13) (12, 22)
Setup (43, 35) 31 44
Run 2
Shall replace here (14, 6) (5, 15)
Removing crossing border edge 5 15 (6, 59, 0)
Setup (3, 19) 53 11
Run 3
Got our guilty line ?
{'ref_node': 1, 'dual': -1, 'candidate': 2, 'n': 0, 'self': <__main__.RGPN object at 0x7f1a5de57828>}
Shall replace here (9, 8) (0, 9)
Setup (12, 20) 10 65
Run 4
Shall replace here (2, 10) (11, 1)
Setup (22, 21) 12 12
Run 5
Shall replace here (66, 10) (9, 11)
Setup (3, 19) 71 4
Run 6
Shall replace here (10, 11) (9, 2)
Setup (58, 49) 56 42
Run 7
Shall replace here (67, 16) (1, 18)
Setup (59, 58) 45 51
Run 8
Shall replace here (13, 12) (14, 5)
Quasi lonely node {node} with #ngbs = 2
Run 9
Shall replace here (1, 10) (67, 0)
Run 10
Shall replace here (14, 5) (13, 22)
Run 11
Shall replace here (9, 2) (10, 11)
Setup (32, 33) 28 40
Run 12
Shall replace here (22, 14) (13, 15)
Setup (4, 1

IndexError: list index out of range

In [ ]:
for x in [(-1, 117, 1), (117, 118, 0), (118, 47, 0), (-1, 120, 0), (120, 47, 0)]:
    print(gpn.dual(x))

In [ ]:
gpn.G.nodes[9], gpn.G.nodes[8], gpn.G.nodes[0]

In [ ]:
gpn.show_all()

In [ ]:
pos = nx.spring_layout(gpn.G, pos=pos, k=0.1, iterations=10)
gpn.update_all_dist()
gpn.show_dist(pos=pos, figsize=(8, 8))

In [ ]:
gpn.dual((125, 15)), gpn.ngb(15), gpn.ngb(7), gpn.dual((125, 7))

In [ ]:
raise

In [ ]:
list_inner = [gpn.G.degree(n) for n in gpn.G.nodes if not gpn.is_border_node(n)]
list_outer = [gpn.G.degree(n) for n in gpn.G.nodes if gpn.is_border_node(n)]

In [ ]:
plt.hist(list_inner)

In [ ]:
plt.hist(list_outer)

In [ ]:
gpn.show_all()

In [ ]:
raise

In [ ]:
# test 1
def test_func():
    gpn = GrowingPlanarNetwork()
    gpn.init_square(8)
    for i in range(500):
        if p_dupl():
            gpn.duplicate_random_node()
        else:
            gpn.remove_random_node()
    dist = np.array(list(nx.betweenness_centrality(gpn.G).values()))
    return (skew(dist), kurtosis(dist), gpn.G, gpn.D)

for i in range(10):
    S, K, G, D = test_func()
    print(G.size(), D.degree(-1))

In [ ]:
G = gpn.G

In [ ]:
pos = nx.spring_layout(G, k=0.1, iterations=50)
pos = nx.planar_layout(gpn.G)
plt.figure(figsize=(12, 12))
nx.draw_networkx(gpn.G, pos=pos)

In [ ]:
pos = nx.planar_layout(Gstart)
plt.figure(figsize=(12, 12))
nx.draw_networkx(Gstart, pos=pos)

In [ ]:
raise

In [ ]:
gpn.show_all()

In [ ]:
ls_ngb = [len(gpn.ngb(x)) for x in gpn.G.nodes]

In [ ]:
plt.hist(ls_ngb)

In [ ]:
raise

In [ ]:
couple_ngb = [(len(gpn.ngb(x)), x) for x in gpn.G.nodes if len(gpn.ngb(x)) > 4]
couple_ngb

In [ ]:
gpn.stabilize_ngb(10)

In [ ]:
couple_ngb = [(len(gpn.ngb(x)), x) for x in gpn.G.nodes if len(gpn.ngb(x)) > 4]
couple_ngb